In [2]:
"""
让模型输出一个低PPL的句子作为key
让模型输出一个高PPL的句子作为key
"""

from modelzipper.tutils import *
import transformers
import math
import torch

model = transformers.AutoModelForCausalLM.from_pretrained("/vepfs/wcf/G/zecheng/hf_models/llama-2-7b-80k").to('cuda:7')
tokenizer = transformers.AutoTokenizer.from_pretrained("/vepfs/wcf/G/zecheng/hf_models/llama-2-7b-80k")
model.eval()

/opt/conda/envs/pan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


ModelZipper is ready for launch🚀 | Current Version🦄 >>> 0.2.7 <<< | AOE Time🕒 2024-05-08 13:19:42


/opt/conda/envs/pan/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.61it/s]


In [29]:
PROMPT = "\nThe best thing to do in San Francisco is eat a sandwich and sit in Dolores Park on a sunny day.\n"
KEY, VALUE = "\nThe best thing to do in San Francisco is", "eat a sandwich and sit in Dolores Park on a sunny day.\n"
KEY = "This is a passkey"
num_sequence = 2

input_ids = tokenizer(KEY, return_tensors='pt').input_ids.to(model.device)
value_ids_len = tokenizer(VALUE, return_tensors='pt').input_ids.size(-1)
candidates = model.generate(input_ids, num_beams=num_sequence, num_return_sequences=num_sequence, penalty_alpha=0.6, no_repeat_ngram_size=2, temperature=0.7, top_p=0.95, do_sample=True, min_new_tokens=value_ids_len, max_new_tokens=value_ids_len + 32, return_dict_in_generate=True, output_scores=True)

sequences_scores = candidates['sequences_scores']
outputs = candidates.sequences[0]

output_str = tokenizer.decode(outputs)
print("decoding results:")
print(output_str)

new_needle = "The best thing to do in San Francisco is to walk around the city. There are so many things to see and do."
reverse_needle_str = " ".join(new_needle.split()[::-1])
print("reversed decoding results:")
print(reverse_needle_str)

# for score, output in zip(sequences_scores, outputs):
#     res = tokenizer.decode(output)
#     print(f"score: {math.exp(score)} | decode: {res}")

decoding results:
<s> This is a passkey for the door to the basement of the house.
This is the first time I've seen this. I don't know what it is. It's a little bit of a mystery. But I think it might be something to
reversed decoding results:
do. and see to things many so are There city. the around walk to is Francisco San in do to thing best The


In [28]:
new_needle = "The best thing to do in San Francisco is to walk around the city. You will be able to see all the sights and sounds of this amazing place."
tok_new_needle = tokenizer(new_needle, return_tensors='pt').input_ids.to(model.device)
new_needle_ppl = torch.exp(model(tok_new_needle, labels=tok_new_needle).loss)
print(new_needle_ppl)

reverse_needle = " ".join(new_needle.split(" ")[::-1])
reverse_new_needle = tokenizer(reverse_needle, return_tensors='pt').input_ids.to(model.device)
reverse_needle_ppl = torch.exp(model(reverse_new_needle, labels=reverse_new_needle).loss)
print(reverse_new_needle)
print(reverse_needle_ppl)


# 做index level 的 reverse
reverse_needle = " ".join(new_needle.split(" ")[::-1])
reverse_new_needle = tokenizer(reverse_needle, return_tensors='pt').input_ids.to(model.device)
reverse_needle_ppl = torch.exp(model(reverse_new_needle, labels=reverse_new_needle).loss)
print(reverse_new_needle)
print(reverse_needle_ppl)

# 做token ids 的 reverse
flip_tok = torch.flip(tok_new_needle[:, 1:], dims=(1,))
flip_needle_str = tokenizer.decode(flip_tok[0])
flip_needle_ids = tokenizer(flip_needle_str, return_tensors='pt').input_ids.to(model.device)
flip_needle_ppl = torch.exp(model(flip_needle_ids, labels=flip_needle_ids).loss)
print(flip_needle_str)
print(flip_needle_ppl)

tensor(5.3507, device='cuda:7', grad_fn=<ExpBackward0>)
tensor([[    1,  2058, 29889, 21863,   292,   445,   310, 10083,   322,   269,
          5861,   278,   599,  1074,   304,  2221,   367,   674,   887,  4272,
         29889,   278,  2820,  6686,   304,   338,  8970,  3087,   297,   437,
           304,  2655,  1900,   450]], device='cuda:7')
tensor(416.8795, device='cuda:7', grad_fn=<ExpBackward0>)
tensor([[    1,  2058, 29889, 21863,   292,   445,   310, 10083,   322,   269,
          5861,   278,   599,  1074,   304,  2221,   367,   674,   887,  4272,
         29889,   278,  2820,  6686,   304,   338,  8970,  3087,   297,   437,
           304,  2655,  1900,   450]], device='cuda:7')
tensor(416.8795, device='cuda:7', grad_fn=<ExpBackward0>)
. placeing amaz this of sounds andights s the all see to able be will You. city the around walk to is Francisco San in do to thing best The
tensor(1476.7153, device='cuda:7', grad_fn=<ExpBackward0>)


In [26]:
print(tok_new_needle)
flip_tok = torch.flip(tok_new_needle[:, 1:], dims=(1,))
flip_str = tokenizer.decode(flip_tok[0])
print(flip_str)

tensor([[    1,   450,  1900,  2655,   304,   437,   297,  3087,  8970,   338,
           304,  6686,  2820,   278,  4272, 29889,   887,   674,   367,  2221,
           304,  1074,   599,   278,   269,  5861,   322, 10083,   310,   445,
         21863,   292,  2058, 29889]], device='cuda:7')
. placeing amaz this of sounds andights s the all see to able be will You. city the around walk to is Francisco San in do to thing best The
